## Requirements

In [ ]:
# For those who, like me, immediately open the code :)
!pip3 install -r -U requirements.txt

## Training

In [ ]:
# to download a dataset from kaggle, can use opendatasets (lib will even unzip it right away)
# my kaggle datasets at the link 😊👉  https://www.kaggle.com/danilalekseev/datasets
import opendatasets as od

od.download("your link")

In [ ]:
# load the required libraries
from loguru import logger
from datetime import datetime
import requests
import pathlib
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import torchvision
import torchvision.transforms as transforms
from torchvision import datasets

In [ ]:
# config logs
logs_path = f"./logs/torhok-{datetime.now()}.log"
logger.add(logs_path, format="{time} - {level} - {message}")
logger.info(f"All logs are saved to the file \"{logs_path}\"")

In [ ]:
data_dir = './data/' # specify the path to the dataset
train_dir = data_dir + '/train'
test_dir = data_dir + '/test'
val_dir = data_dir + '/val'

batch_size = 64
num_epochs = 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classes = sorted([_.name.split("/")[-1] for _ in pathlib.Path(train_dir).iterdir()])

train_count = len(glob.glob(train_dir + '/**/*.*'))
test_count = len(glob.glob(test_dir + '/**/*.*'))
val_count = len(glob.glob(val_dir + '/**/*.*'))


mes = f"All parameters have been declared\n\t\"{device}\" device detected\n\t" +\
f"classes = {', '.join(classes)}\n\t/train {train_count}\n\t/test {test_count}\n\t" +\
f"/val {val_count}\n\tbatch_size = {batch_size}"
logger.info(mes)

In [ ]:
transform = transforms.Compose(
    [transforms.Resize((150, 150)),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize([0.5, 0.5, 0.5], 
                          [0.5, 0.5, 0.5])])

# loading Datasets with ImageFolder
trainset = datasets.ImageFolder(train_dir, transform=transform)
valset = datasets.ImageFolder(val_dir, transform=transform)
testset = datasets.ImageFolder(test_dir, transform=transform)

# using image and form datasets, define a data loader.
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size)

logger.info("Images have been uploaded and transformed")

In [ ]:
#CNN Network

class ConvNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNet, self).__init__()
        
        # Output size after convolution
        
        self.conv1=nn.Conv2d(in_channels=3, 
                             out_channels=12, 
                             kernel_size=3, 
                             stride=1, 
                             padding=1)
        self.bn1=nn.BatchNorm2d(num_features=12)
        self.relu1=nn.ReLU()
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        
        
        self.conv2=nn.Conv2d(in_channels=12,
                             out_channels=20,
                             kernel_size=3,
                             stride=1,
                             padding=1)
        self.relu2=nn.ReLU()    
        
        self.conv3=nn.Conv2d(in_channels=20,
                             out_channels=32,
                             kernel_size=3,
                             stride=1,
                             padding=1)
        self.bn3=nn.BatchNorm2d(num_features=32)
        self.relu3=nn.ReLU()
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,
                          out_features=num_classes)
        
    # Direct feed function
    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
            
        output = self.pool(output)
            
        output = self.conv2(output)
        output = self.relu2(output)
            
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
            
        output=output.view(-1,32*75*75)
        output=self.fc(output)

        return output

model = ConvNet(num_classes=len(classes)).to(device)

logger.info("Model architecture:\n\n" + str(model)+"\n\n")
logger.info("CNN model declared successfully")

In [ ]:
# Optimizer & loss function
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
loss_function = nn.CrossEntropyLoss()

logger.info("Optimizer & loss function declared successfully")

In [ ]:
# Model training
def train_classifier(epochs, print_every_iterration=100, 
                     save_mode_path="./torch_model.pth"):
    logger.debug(f"Start training the model. Number of all epochs - {num_epochs}")
    best_accuracy = 0.0

    for epoch in range(epochs):
        model.train()
        train_accuracy = 0.0
        train_loss = 0.0

        for i, (images, labels) in enumerate(trainloader):
            if i % print_every_iterration == 0:
                logger.debug(f"Iteration: {i}/{len(trainloader)}")

            if torch.cuda.is_available():
                images = Variable(images.cuda())
                labels = Variable(labels.cuda())
                
            optimizer.zero_grad()
            
            outputs = model(images)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()
            
            
            train_loss += loss.cpu().data * images.size(0)
            _, prediction = torch.max(outputs.data,1)
            
            train_accuracy += int(torch.sum(prediction == labels.data))
        logger.info(f"All iterations of epoch {epoch}/{epochs} have been completed")
            
        train_accuracy = train_accuracy / train_count
        train_loss = train_loss / train_count
        
        
        # Evaluation on validation dataset
        model.eval()
        
        val_accuracy = 0.0
        for i, (images, labels) in enumerate(valloader):
            if torch.cuda.is_available():
                images = Variable(images.cuda())
                labels = Variable(labels.cuda())
                
            outputs = model(images)
            _, prediction = torch.max(outputs.data, 1)
            val_accuracy += int(torch.sum(prediction == labels.data))
        
        val_accuracy = val_accuracy / val_count
        
        
        logger.debug(f'Epoch: {epoch}/{epochs}  Train Loss: {train_loss} Train Accuracy: {train_accuracy} Validation Accuracy: {val_accuracy}')
        
        # Save the best result of the model
        if val_accuracy > best_accuracy:
            torch.save(model.state_dict(), save_mode_path)
            best_accuracy = val_accuracy

        
        logger.info(f"Model training was successful\nThe best result of the model has been saved in \"{save_mode_path}\"")


train_classifier(epochs=num_epochs, print_every_iterration=10)

# Testing

In [ ]:
# load the required libraries
from torchmetrics.functional import accuracy, precision_recall
from torchmetrics import F1Score, MeanSquaredError
from PIL import Image
import random
import glob
import pathlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
model_path = "./torch_model.pth" # path to pytorch model
images_info = {"path": "./data/test/", "count": 3} # path where the photos for tests are located & аnd the quantity to be used
classes = sorted([_.name.split("/")[-1] for _ in pathlib.Path(images_info["path"]).iterdir()])

In [ ]:
# loading the saved pth model
model = ConvNet(num_classes=len(classes))
model.load_state_dict(torch.load(model_path))
model.eval()

In [ ]:
transform = transforms.Compose([transforms.Resize((150, 150)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
# get all the paths to the images, mix and take part
testimgs = glob.glob(images_info["path"] + '**/*.*')
random.shuffle(testimgs)
testimgs = testimgs[:images_info["count"]]

lables = "true lables: " + " ".join([_.replace(images_info["path"], "").split("/")[0] for _ in testimgs])
images = [[transform(Image.open(_)).numpy()] for _ in testimgs]

images = torch.tensor(np.vstack(images))

# displaying random images
def imgshow(img):
    img = img / 2 + 0.5 # unnormalize
    plt.imshow(np.transpose(img, (1, 2, 0)))
    plt.show()

imgshow(torchvision.utils.make_grid(images))

print(lables)

In [ ]:
# get the prediction of the neural network model
outputs = model(images)
_, predicted = torch.max(outputs, 1)

print('predicted: ', ' '.join(f'{classes[predicted[j]]:5s}' for j in range(images_info["count"])))

In [ ]:
# collect information, y_true - correct data, y_pred - neural network predictions (takes time)
y_true = []
x_test = []
print("Number of images", len(glob.glob(images_info["path"] + '**/*.*')))

for imgpath in glob.glob(images_info["path"] + '**/*.*'):
    try:
        y_true.append(classes.index(imgpath.replace(images_info["path"], "").split("/")[0]))
        x_test.append([transform(Image.open(imgpath)).numpy()])
    except Exception as error: # skip if an error occurred while processing the image
        # print(repr(error), "- passing img")
        pass

y_true = torch.tensor(y_true)
y_pred = torch.max(model(torch.tensor(np.vstack(x_test))), 1)[1]

In [ ]:
# get neural network quality metrics
def metrics(y_true, y_pred):
    # calculate the Accuracy
    accurancy = accuracy(y_pred, y_true)
    print("Accurancy:", round(float(accurancy)*100, 3), "%")
    
    # calculate the Precision & Recall
    precision, recall = map(float, precision_recall(y_pred, y_true))
    print("Precision:", round(precision*100, 3), "%")
    print("Recall:", round(recall*100, 3), "%")
    
    # calculate the F1Score
    f1 = F1Score(num_classes=len(classes))
    f1score = f1(y_pred, y_true)
    print("F1Score:", round(float(f1score)*100, 3), "%")
    
    # calculate the Mean Squared Error
    mse = MeanSquaredError()
    mean_squared_error = mse(y_pred, y_true)
    print("Mean Squared Error:", round(float(mean_squared_error)*100, 3), "%")

metrics(y_true=y_true, y_pred=y_pred)